In [1]:
import random

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from string import punctuation, printable
import re

import spacy
nlp = spacy.load("en_core_web_sm")


import sklearn

import torch
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)

cpu


In [2]:
from distutils.dir_util import copy_tree

copy_tree('/kaggle/input/spellchecker', '/kaggle/working/')

!gzip '/kaggle/working/spellchecker/resources/en.json'

!pip install '/kaggle/working/pyspellchecker-0.8.1-py3-none-any.whl'

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Processing ./pyspellchecker-0.8.1-py3-none-any.whl


In [3]:
from spellchecker import SpellChecker

spell = SpellChecker()

In [4]:
input_dir = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2"
train_data = pd.read_csv(os.path.join(input_dir, 'train.csv'))
test_data = pd.read_csv(os.path.join(input_dir, 'test.csv'))
extra_data = pd.read_csv('/kaggle/input/persaude-corpus-2/persuade_2.0_human_scores_demo_id_github.csv')

extra_data = extra_data.rename(columns={'essay_id_comp': 'essay_id',
                                        'holistic_essay_score': 'score'})
extra_data = extra_data[['essay_id', 'full_text', 'score']]

extra_data = extra_data[~extra_data['full_text'].isin(train_data['full_text'])]

print(train_data.shape, test_data.shape, extra_data.shape)

train_data.sample(5)

(17307, 3) (3, 2) (13125, 3)


,essay_id,full_text,score
12696,bb4c434,"People tend to use there cars so much, they ba...",3
4625,44e88b0,Imagine being a top scientist at NASA and Viki...,3
733,0ba78ec,The face of Mars could not be created by alien...,3
16885,f96c287,Many people belive that the face on Mars was c...,3
3334,317173f,Driverless Cars are coming soon or later? Peop...,4


In [5]:
def count_misspelled(words):
    words = [word for word in words if word.strip()]

    words = set(words) - {"'s", "n't"} - set(punctuation)

    return spell.unknown(words)


def get_sentences(text: str):
    text = "".join(filter(lambda x: x in printable, text)).strip()
    doc = nlp(text)
    return [sent.text for sent in doc.sents]

def preprocess_text(text: str):
    text = re.sub(r'\s+', ' ', text)
    text = "".join(filter(lambda x: x in printable, text)).strip()
    return text


def get_features(data_df: pd.DataFrame, output_dir='./output/data_features.csv', save=True):
    # parse the text into paragraphs
    data_df['paragraphs'] = data_df['full_text'].apply(lambda x: x.split("\n\n"))
    data_df['num_paragraphs'] = data_df['paragraphs'].apply(len)

    data_df['full_text'] = data_df['full_text'].apply(preprocess_text)

    # into sentences
    data_df['sentences'] = data_df['full_text'].apply(get_sentences)
    data_df['num_sentences'] = data_df['sentences'].apply(len)

    # Create lists to hold the results
    words = []
    lemmas = []
    pos = []
    is_stop_word = []

    # Process the texts in batches
    for i, doc in enumerate(nlp.pipe(data_df['full_text'], batch_size=50)):
        print(f"Processing batch {i+1}/{len(data_df)}", end="\r")

        words.append([token.text for token in doc])
        lemmas.append([token.lemma_ for token in doc])
        pos.append([token.pos_ for token in doc])
        is_stop_word.append([token.is_stop for token in doc])

    data_df['words'] = words
    data_df['lemma'] = lemmas
    data_df['pos'] = pos
    data_df['is_stop_word'] = is_stop_word

    data_df['num_words'] = data_df['words'].apply(len)

    data_df['num_conjunctions'] = data_df['pos'].apply(
        lambda x: len([pos for pos in x if pos == 'CCONJ']))

    data_df['num_distinct_words'] = data_df['lemma'].apply(
        lambda x: len(set(x)))

    data_df['num_misspell'] = data_df['lemma'].apply(count_misspelled)
    data_df['num_misspell'] = data_df['num_misspell'].apply(len)

    data_df['mean_word_len'] = data_df['lemma'].apply(lambda x: np.mean(
        [len(word) for word in x if word.strip() and word not in punctuation]))

    data_df['mean_sent_len'] = data_df['sentences'].apply(
        lambda x: np.mean([len([token.text for token in nlp(sent)]) for sent in x]))

    if save:
        data_df.to_csv(output_dir, index=False)

    return data_df

test_df = get_features(test_data)

In [7]:
model_path = '/kaggle/input/essay-scoring-models/longformer-base-4096'
tokenizer = AutoTokenizer.from_pretrained(model_path)
embedder = AutoModel.from_pretrained(model_path)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
ling_features = ['num_paragraphs', 'num_words', 'num_conjunctions',
                 'num_distinct_words', 'num_misspell',
                 'mean_word_len', 'num_sentences', 'mean_sent_len']

hyperparameters = {
    'lr': 1e-4,
    'dropout': 0.2,
    'epochs': 10,
    'batch_size': 64,
    'test_set': {
        'total': len(test_df),
    },
    'linguistic_features': ling_features,
    'accelator': str(device)
}

In [8]:
ling_features = ['num_paragraphs', 'num_words', 'num_conjunctions',
                 'num_distinct_words', 'num_misspell',
                 'mean_word_len', 'num_sentences', 'mean_sent_len']

In [9]:
test_tokenized = tokenizer(test_df['full_text'].tolist(),
                           max_length=hyperparameters['max_seq_len'], 
                           padding='max_length', truncation=True, 
                           return_tensors="np")

test_df['input_ids'] = test_tokenized['input_ids'].tolist()
test_df['attention_mask'] = test_tokenized['attention_mask'].tolist()

print(len(test_df.sample(1).iloc[0]['input_ids']))

/tmp/ipykernel_18/3159259489.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['input_ids'] = train_tokenized['input_ids'].tolist()
/tmp/ipykernel_18/3159259489.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['attention_mask'] = train_tokenized['attention_mask'].tolist()


2560
2560


/tmp/ipykernel_18/3159259489.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['input_ids'] = val_tokenized['input_ids'].tolist()


2560
2560


/tmp/ipykernel_18/3159259489.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['attention_mask'] = val_tokenized['attention_mask'].tolist()


In [12]:
class MultiFeaturesModel(torch.nn.Module):
    def __init__(self, embedder,
                 lf_input_size, lf_hidden_size=64,
                 dropout=0.2):
        super(MultiFeaturesModel, self).__init__()
        # freeze
        for param in embedder.parameters():
            param.requires_grad = False
            
        self.embedder = embedder
        self.lf = torch.nn.Linear(lf_input_size,lf_hidden_size)
        self.fc = torch.nn.Linear(lf_hidden_size + embedder.config.hidden_size, 128)
        self.regressor = torch.nn.Linear(128, 1)
        self.dropout = torch.nn.Dropout(dropout)
    
    def config(self):
        return {
            'embedder': self.embedder.config,
            'lf': {
                'input_size': self.lf.in_features,
                'hidden_size': self.lf.out_features
            },
            'fc': {
                'input_size': self.fc.in_features,
                'hidden_size': self.fc.out_features
            },
            'regressor': {
                'input_size': self.regressor.in_features,
                'output_size': self.regressor.out_features
            }
        }

    def forward(self, token_ids, attention_mask, ling_features):
        embedded = self.embedder(token_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        if self.training:
            embedded = self.dropout(embedded)
            
        ling_features = F.leaky_relu(self.lf(ling_features))
        if self.training:
            ling_features = self.dropout(ling_features)
            
        features = torch.cat((embedded, ling_features), dim=1)
        features = F.leaky_relu(self.fc(features))
        if self.training:
            features = self.dropout(features)
            
        score = self.regressor(features)
        return score


model = MultiFeaturesModel(embedder, len(ling_features),
                           128,
                           hyperparameters['dropout'])

model.load_state_dict(torch.load('/kaggle/input/essay-scoring-models/checkpoints/multi_features_longformer-base-4096_2024Y-05m-04d_model.pth',
                                 map_location=device))

In [ ]:
def logit_to_score(logit, min_score=1, max_score=6):
    scores = torch.clamp(torch.round(logit), min_score, max_score)
    scores = scores.long()
    return scores

In [ ]:
model.eval()
test_predictions = []

with torch.no_grad():
    for row in test_df.itertuples():
        token_ids = torch.tensor(row.input_ids).unsqueeze(0)
        attention_mask = torch.tensor(row.attention_mask).unsqueeze(0)
        features = torch.tensor([row[1:11]]).float()
        output = model(token_ids.to(device), 
                       attention_mask.to(device), 
                       features.to(device))
        test_predictions.append(output.cpu().numpy()[0][0])

submit_df = pd.DataFrame({
    'essay_id': test_df['essay_id'],
    'prediction': logit_to_score(test_predictions)
})
submit_df.to_csv('submission.csv', index=False)